# Case 3: The Gentle Lentil Restaurant
## STAT 5605: Decision Models

### Introduction

In this tutorial, we will use following case from the book *Data, Models, and Decision: the Fundamentals of Management Science* by Bertsimas and Freund to understand the basic concepts of Monte Carl Simulation.  In the case, we are presented with the story of Sanjay Thomas, a second-year MBA student at MIT's Sloan School of Management.  He is in the midst of a dilemma of having to choose between a job offer with a top management consulting firm and the possibility of opening his own restaurant - something he has always wanted to do.  After much consideration, Sanjay determines that the lifestyle tradeoffs between the two options are comparable, and the decision will ultimately come down to whichever provides the better income.

The offer from the management consulting firm is straightforward, `$`80,000 for the first year with the firm.  Sanjay finds that the earnings potential of opening his own restaurant, however, is much more complicated.  Sanjay has put a lot of thought and study into the revenue potential of a restaurant like the one he is considering, but due to the many factors involved, it is impossible to simply calculate the expected revenue/profit.  To get a good understanding of the plausible income from this venture, Sanjay must turn to simulation methods so that he can combine the various distributions of cost and revenue to determine a feasible range for the income of his restaurant concept.  
  
### Solving Sanjay's Dilemma 

According to [Wikipedia](https://en.wikipedia.org/wiki/Monte_Carlo_method):
>**Monte Carlo methods** (or **Monte Carlo experiments**) are a broad class of computational algorithms that rely on repeated random sampling to obtain numerical results. ... Monte Carlo methods are mainly used in three distinct problem classes:<sup id="cite_ref-1" class="reference"><a href="#cite_note-1">[1]</a></sup> <a href="/wiki/Optimization" class="mw-redirect" title="Optimization">optimization</a>, <a href="/wiki/Numerical_integration" title="Numerical integration">numerical integration</a>, and generating draws from a <a href="/wiki/Probability_distribution" title="Probability distribution">probability distribution</a>. 

In this case, the problem class that we're primarily interested in is generating draws from a probability distribution.  We will use Monte Carlo methods, implemented in Python, to answer the questions from the book.  Along the way, we'll learn how to code these techniques in Python.  As we will see, Monte Carlo is an extremely powerful (and surprisingly simple) simulation method that can help us to make well-informed decisions in the midst of great uncertainty.

Before we dive into the code, let's talk about the problem as it's described in the book.  From the case, we know that Sanjay's goal is to determine whether he'll make more money by a) accepting the job offer from the consulting firm (`$`80,000 in the first year) or b) opening his own restaurant, The Gentle Lentil, in Boston's Harvard Square.  To compare the two options, we need to determine the profit Sanjay can reasonably expect from the first year in business at The Gentle Lentil.  We know formuola for profit is simply $profit = revenue - expenses$.  The problem is that number of sales, sales price, and labor costs of the restaurant are all variable and have different associated distributions.  Fortunately, Sanjay has done a lot of research and is able to provide us with the appropriate distribution for each variable.  

Knowing these distributions, we can use Monte Carlo to create a confidence interval around Sanjay's expected profit for the first year which we can then compare to the offer of `$`80,000 he has received from the consulting firm.  The general process of a Monte Carlo simulation in this situation is:


1. Sample from each distribution
2. Combine the samples with the profit formula
3. Repeat steps 1 & 2 twelve times (12 months in a year)
4. Sum across all months to determine the profit for that year

We can perform the above process many times to get the shape of the distribution for Sanjay's expected profit in the first year of business.  It's important to note, however, that the distribution obtained from our simulation may still be hard to compare with the consulting firm's offer, but, as we'll see, there are other statistical techniques that we can use to help us compare more easily between the two options.  

### By the Numbers

The key to setting up our model is understanding the distributions surrounding our variables and how to sample from them.  As we've already seen, Sanjay has determined that the most important variables are:

**- The number of meals sold per month**

We are told that the number of meals sold per month can be given as a Normal distribution with a mean of 3,000 and a standard deviation of 1,000.  

**- Revenue per meal**

Sanjay has also determined that he will offer a prix fixe menu and that the price would be set on his estimate of the local economy.  Here is the table describing this distribution:

|Scenario|Prix Fixe Meal Price|Probability|
|--------|--------------------|-----------|
|Very healthy market| `$`20.00| 25% |
|Healthy market| `$`18.50| 35% |
|Not so healthy market| `$`16.50| 30% |
|Unhealthy market| `$`15.00| 10% |

**- Labor costs of the restaurant**

Sanjay has estimated the labor cost of the restaurant to be between `$`5,040 and `$`6,860 per month.  He has assumed that the actual cost of labor will form a uniform distribution within this range.

Additionally, Sanjay has determined that there will be a variable cost of `$`11 per meal served.  There is also a fixed operating cost (rent, equipment lease, etc.) of `$`3,995 per month.

Using all of this information, we can rewrite our profit formula for each month as:

$$ profit = (N(3000, 1000) * F(p)) - (3995 + U(5040, 6860) + (N * 11)) $$

Where $N(3000, 1000)$ is a normal distribution with mean of 3,000 and standard devation of 1000 (the number of meals sold in a month), $F(p)$ is a selection from the probability table above (the price of each meal), $U(5040, 6860)$ is a selection from a uniform distribution with the given range (labor cost per month) and $N * 11$ represents the variable cost associated with the number of meals sold in the month.

Now that we've sorted out our variables, let's see how to draw samples from each of them, using Python and Numpy,  to arrive at our profit for each month.  Thanks to the power of Numpy`*` simulating the number of meals sold per month and labor costs are rather trivial.

`*`All of these functions can be performed with the Python's standard library *random*, but we're going to use some other functionality of Numpy later on to speed up our simulation.

In [3]:
# Library Imports
import numpy as np

# Number of Meals Sold per Month
def num_meals_sold(N=1):
    """
    Obtain a random sample from a normal distribution with mean 3000 and standard deviation 1000. 
    This represents the number of meals sold in a month.
    
    N = number of months to sample
    """
    
    return np.random.normal(3000, 1000, N)

# Labor Cost per Month
def labor_cost(N=1):
    """
    Obtain a random sample from a uniform distribution in the range (5040, 6860), representing 
    labor cost per month
    
    N = number of months to sample
    """
    
    return np.random.random(5040, 6860)


Above, we used some convenience functions from Numpy to sample from normal and uniform distributions which makes the process of selecting from these distributions rather trivial.  However, there is no convenience function that allows us to sample from a discrete distribution (so far as I am aware), so this makes the process of sampling the price of each meal in a month somewhat more complicated.  Thankfully, the standard uniform distribution (0, 1) has some unique properties we can use to recreate the discrete probability distribution of meal price from above.

In order recreate this distribution, we will turn to the definition of a uniform distribution.  According to [Wikipedia](https://en.wikipedia.org/wiki/Uniform_distribution_(continuous%29):

> In <a href="/wiki/Probability_theory" title="Probability theory">probability theory</a> and <a href="/wiki/Statistics" title="Statistics">statistics</a>, the <b>continuous uniform distribution</b> or <b>rectangular distribution</b> is a family of <a href="/wiki/Symmetric_distribution" class="mw-redirect" title="Symmetric distribution">symmetric</a> <a href="/wiki/Probability_distributions" class="mw-redirect" title="Probability distributions">probability distributions</a> such that for each member of the family, all <a href="/wiki/Interval_(mathematics)" title="Interval (mathematics)">intervals</a> of the same length on the distribution's support are equally probable. 